# Semantyczne Jądro z integracją serwera OpenBnB MCP

Ten notatnik pokazuje, jak używać Semantycznego Jądra z rzeczywistym serwerem OpenBnB MCP do wyszukiwania prawdziwych ofert Airbnb za pomocą MCPStdioPlugin. Do dostępu do LLM wykorzystuje Azure AI Foundry. Aby skonfigurować zmienne środowiskowe, możesz postępować zgodnie z [lekcją konfiguracji](/00-course-setup/README.md).


## Zaimportuj potrzebne pakiety


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Tworzenie połączenia wtyczki MCP

Połączymy się z [serwerem MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) za pomocą MCPStdioPlugin. Ten serwer udostępnia funkcję wyszukiwania Airbnb za pośrednictwem pakietu @openbnb/mcp-server-airbnb.


## Tworzenie klienta

W tym przykładzie użyjemy Azure AI Foundry do uzyskania dostępu do LLM. Upewnij się, że zmienne środowiskowe są poprawnie skonfigurowane.


## Konfiguracja środowiska

Skonfiguruj ustawienia Azure OpenAI. Upewnij się, że masz ustawione następujące zmienne środowiskowe:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Zrozumienie integracji OpenBnB MCP

Ten notatnik łączy się z **prawdziwym serwerem OpenBnB MCP**, który zapewnia rzeczywistą funkcjonalność wyszukiwania Airbnb.

### Jak to działa:

1. **MCPStdioPlugin**: Wykorzystuje komunikację za pomocą standardowego wejścia/wyjścia z serwerem MCP  
2. **Prawdziwy pakiet NPM**: Pobiera i uruchamia `@openbnb/mcp-server-airbnb` za pomocą npx  
3. **Dane na żywo**: Zwraca rzeczywiste dane o nieruchomościach Airbnb z ich API  
4. **Odkrywanie funkcji**: Agent automatycznie wykrywa dostępne funkcje serwera MCP  

### Dostępne funkcje:

Serwer OpenBnB MCP zazwyczaj udostępnia:
- **search_listings** - Wyszukiwanie nieruchomości Airbnb według lokalizacji i kryteriów  
- **get_listing_details** - Pobieranie szczegółowych informacji o konkretnych nieruchomościach  
- **check_availability** - Sprawdzanie dostępności w określonych terminach  
- **get_reviews** - Pobieranie opinii o nieruchomościach  
- **get_host_info** - Pobieranie informacji o gospodarzach nieruchomości  

### Wymagania wstępne:

- Zainstalowany **Node.js** na Twoim systemie  
- **Połączenie z Internetem**, aby pobrać pakiet serwera MCP  
- Dostępność **NPX** (dołączony do Node.js)  

### Testowanie połączenia:

Możesz przetestować serwer MCP ręcznie, uruchamiając:  
```bash
npx -y @openbnb/mcp-server-airbnb
```

To pobierze i uruchomi serwer OpenBnB MCP, do którego Semantic Kernel następnie się połączy, aby uzyskać rzeczywiste dane z Airbnb.


## Uruchamianie Agenta z serwerem OpenBnB MCP

Teraz uruchomimy Agenta AI, który łączy się z serwerem OpenBnB MCP, aby wyszukać prawdziwe zakwaterowanie Airbnb w Sztokholmie dla 2 dorosłych i 1 dziecka. Możesz swobodnie zmienić listę `user_inputs`, aby dostosować kryteria wyszukiwania.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Podsumowanie  
Gratulacje! Udało Ci się stworzyć agenta AI, który integruje się z rzeczywistym wyszukiwaniem zakwaterowania za pomocą protokołu Model Context Protocol (MCP):  

Technologie użyte:  
- Semantic Kernel - do budowy inteligentnych agentów z Azure OpenAI  
- Azure AI Foundry - do obsługi LLM i uzupełniania rozmów  
- MCP (Model Context Protocol) - do standaryzowanej integracji narzędzi  
- OpenBnB MCP Server - do rzeczywistego wyszukiwania zakwaterowania na Airbnb  
- Node.js/NPX - do uruchamiania zewnętrznego serwera MCP  

Czego się nauczyłeś:  
- Integracja MCP: Łączenie agentów Semantic Kernel z zewnętrznymi serwerami MCP  
- Dostęp do danych w czasie rzeczywistym: Wyszukiwanie rzeczywistych nieruchomości Airbnb za pomocą API na żywo  
- Komunikacja protokołu: Korzystanie z komunikacji stdio między agentem a serwerem MCP  
- Odkrywanie funkcji: Automatyczne wykrywanie dostępnych funkcji z serwerów MCP  
- Strumieniowe odpowiedzi: Rejestrowanie i logowanie wywołań funkcji w czasie rzeczywistym  
- Renderowanie HTML: Formatowanie odpowiedzi agenta w stylizowanych tabelach i interaktywnych wyświetleniach  

Kolejne kroki:  
- Zintegruj dodatkowe serwery MCP (pogoda, loty, restauracje)  
- Zbuduj system multi-agentowy łączący protokoły MCP i A2A  
- Stwórz własne serwery MCP dla swoich źródeł danych  
- Zaimplementuj trwałą pamięć rozmów między sesjami  
- Wdróż agenta w Azure Functions z orkiestracją serwera MCP  
- Dodaj uwierzytelnianie użytkownika i funkcje rezerwacji  

Kluczowe zalety architektury MCP:  
- Standaryzacja: Uniwersalny protokół do łączenia agentów AI z zewnętrznymi narzędziami  
- Dane w czasie rzeczywistym: Dostęp do aktualnych informacji z różnych usług  
- Rozszerzalność: Łatwa integracja nowych źródeł danych i narzędzi  
- Interoperacyjność: Działa w różnych ramach AI i platformach agentów  
- Rozdzielenie obowiązków: Wyraźne rozróżnienie między logiką AI a dostępem do danych zewnętrznych  



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić poprawność tłumaczenia, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
